In [3]:
!pip install webdriver-manager selenium

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

cdm = ChromeDriverManager().install()
s=Service(cdm)
driver = webdriver.Chrome(service=s)
driver.maximize_window()
driver.get('https://www.google.com')
driver.find_element(By.NAME, 'q').send_keys('Testing')

[WDM] - 

[WDM] - ====== WebDriver manager ======


ValueError: Could not get version for google-chrome with the command: google-chrome --version || google-chrome-stable --version